In [1]:
import cv2
import numpy as np
import tensorflow as tf
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

# The function below takes the input from the user and create the tracker if required as per user's choice

In [2]:
def ask_for_input():
    print("1. object detection")
    print("2. object tracting")
    print("3. both object detection and tracking")
    choice=input("select any of the beow options")
    if choice=="2" or choice=="3":
        print("1. boosting")
        print("2. MIL")
        print("3. KCF")
        print("4. TLD")
        print("5. MEDIANFLOW")
        print("6. GOTURN")
        print("7. MOSSE")
        print("8. CSRT")
        tracker_name=input("please select your tracker")
        if choice=='1':
            tracker=cv2.TrackerBoosting_create()
        elif choice=='2':
            tracker=cv2.TrackerMIL_create()
        elif choice=='3':
            tracker=cv2.TrackerKCF_create()
        elif choice=='4':
            tracker=cv2.TrackerTLD_create()
        elif choice=='5':
            tracker=cv2.TrackerMedianFlow_create()
        elif choice=='6':
            tracker=cv2.TrackerGOTURN_create()
        elif choice=='7':
            tracker=cv2.TrackerMOSSE_create()
        elif choice=='8':
            tracker=cv2.TrackerCSRT_create()
        else:
            tracker=cv2.TrackerBoosting_create()   
    else:
        tracker="0"
    return choice,tracker

# The below function is called when the user needs only "object detection"

In [3]:
def object_detection():
    if tf.__version__ < '1.10.0':
        raise ImportError('Please upgrade your tensorflow installation to v1.10.* or later!')
    PATH_TO_CKPT = './frozen_inference_graph.pb'
    font = cv2.FONT_HERSHEY_SIMPLEX
    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')
    THRESHOLD = 0.6  #  threshold for detections i.e.,we are considering the objects only if the score of the object is above 60% 
    PATH_TO_LABELS = os.path.join('mscoco_label_map.pbtxt')
    NUM_CLASSES = 90
    label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
    categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    with detection_graph.as_default():
        with tf.Session(graph=detection_graph) as sess:
            #provide the ip address that is shown in your mobile when you start the video stream in the app
            cap = cv2.VideoCapture("http://192.168.1.6:8080/video")
            #comment the above line and uncomment the below line if web cam is used 
            #cap=cv2.VideoCapture(0)
            # Check if the app in the mobile is opened correctly and mobile is connected to same wifi as of your machine
            if not cap.isOpened():
                raise IOError("Cannot connect")
            # Definite input and output Tensors for detection_graph
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            # Each box in the image represents a particular object that is detected.
            detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            # Score represents percentage of confidence that model have for each of the objects.
            # Images only above 60% score are shown to the user
            # score value is shown on the result image, together with the class label.
            detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
            detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            ret, frame = cap.read()
            height, width, _ = frame.shape
            while True:
                ret, frame = cap.read()
                (boxes, scores, classes, num) = sess.run(
                  [detection_boxes, detection_scores, detection_classes, num_detections],
                  feed_dict={image_tensor: np.expand_dims(frame, axis=0)})
                for i, box in enumerate(boxes[0]):
                    if scores[0][i] > THRESHOLD:
                        x = box[1] * width
                        y = box[0] * height
                        right = box[3] * width
                        bottom = box[2] * height
                        frame = cv2.rectangle(frame,(int(x),int(y)),(int(right),int(bottom)),(0,0,255),2)
                        vis_util.visualize_boxes_and_labels_on_image_array(frame,np.squeeze(boxes),np.squeeze(classes).astype(np.int32),np.squeeze(scores),category_index,use_normalized_coordinates=True,line_thickness=8)

                cv2.imshow('object_detection', frame)

                # Press esc to exit
                if cv2.waitKey(50)&0xFF==27:
                    break
        cap.release()
        cv2.destroyAllWindows()

# The below function is called when the user needs object tracking

In [4]:
def object_tracking(tracker):
    tracker_name=str(tracker).split()[0][1:]
    #provide the ip address that is shown in your mobile when you start the video stream in the app
    cap = cv2.VideoCapture("http://192.168.1.6:8080/video")
    #comment the above line and uncomment the below line if web cam is used 
    #cap=cv2.VideoCapture(0)
    ret,frame=cap.read() #read the first frame
    roi=cv2.selectROI(frame,False) #select ROI
    ret=tracker.init(frame,roi) #initializing tracker
    mask=np.zeros_like(frame)
    color=(0,255,0)
    temp=(0,0)
    while True:
        ret,frame=cap.read() 
        success,roi=tracker.update(frame) #updating the tracker
        (x,y,w,h)=tuple(map(int,roi)) 
        #drawing rectangle as tracker moves
        if success:
            pts1=(x,y)
            pts2=(x+w,y+h)
            cv2.rectangle(frame,pts1,pts2,(255,125,25),3)
            mask=cv2.line(mask,pts1,temp,color,2)
            frame=cv2.circle(frame,temp,3,color,-1)
            frame=cv2.add(frame,mask)
            temp=pts1
        else:
            cv2.putText(frame,"fail to track the object",(100,200),cv2.FONT_HERSHEY_SIMPLEX,1,(25,125,255),3)
        cv2.putText(frame,tracker_name,(20,400),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,0),3)
        cv2.imshow("object_tracking",frame)
        if cv2.waitKey(50)&0xFF==27:
            break
    cap.release()
    cv2.destroyAllWindows()

# The below function is called when the user needs both object detection and tracking

In [5]:
def object_detection_tracking(tracker):
    tracker_name=str(tracker).split()[0][1:] 
    if tf.__version__ < '1.10.0':
        raise ImportError('Please upgrade your tensorflow installation to v1.10.* or later!')
    PATH_TO_CKPT = './frozen_inference_graph.pb'
    font = cv2.FONT_HERSHEY_SIMPLEX
    detection_graph = tf.Graph()
    with detection_graph.as_default():
        od_graph_def = tf.GraphDef()
        with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
            serialized_graph = fid.read()
            od_graph_def.ParseFromString(serialized_graph)
            tf.import_graph_def(od_graph_def, name='')
    THRESHOLD = 0.6  #threshold for detections i.e., we are considering the objects only if the score of the object is above 60% 
    PATH_TO_LABELS = os.path.join('mscoco_label_map.pbtxt')
    NUM_CLASSES = 90
    label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
    categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    with detection_graph.as_default():
        with tf.Session(graph=detection_graph) as sess:
            #provide the ip address that is shown in your mobile when you start the video stream in the app
            cap = cv2.VideoCapture("http://192.168.1.6:8080/video")
            #comment the above line and uncomment the below line if web cam is used 
            #cap=cv2.VideoCapture(0)
            # Check if the app in the mobile is opened correctly and mobile is connected to same wifi as of your machine
            if not cap.isOpened():
                raise IOError("Cannot connect")
            # Definite input and output Tensors for detection_graph
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            # Each box in the image represents a particular object that is detected.
            detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            # Score represents percentage of confidence that model have for each of the objects.
            # Images only above 60% score are shown to the user
            # score value is shown on the result image, together with the class label.
            detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
            detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            ret, frame = cap.read() 
            # frame is stored in temp_frame for further use in object tracking part as there are changes done to frame while object detection 
            temp_frame=frame 
            roi=cv2.selectROI(temp_frame,False) #select ROI
            ret=tracker.init(temp_frame,roi) #initializing tracker
            mask=np.zeros_like(temp_frame)
            color=(0,255,0)
            temp=(0,0)
            height, width, _ = frame.shape
            while True:
                ret, frame = cap.read()
                temp_frame=frame
                (boxes, scores, classes, num) = sess.run(
                  [detection_boxes, detection_scores, detection_classes, num_detections],
                  feed_dict={image_tensor: np.expand_dims(frame, axis=0)})
                for i, box in enumerate(boxes[0]):
                    if scores[0][i] > THRESHOLD:
                        x = box[1] * width
                        y = box[0] * height
                        right = box[3] * width
                        bottom = box[2] * height
                        frame = cv2.rectangle(frame,(int(x),int(y)),(int(right),int(bottom)),(0,0,255),2)
                        vis_util.visualize_boxes_and_labels_on_image_array(frame,np.squeeze(boxes),np.squeeze(classes).astype(np.int32),np.squeeze(scores),category_index,use_normalized_coordinates=True,line_thickness=8)
                ### End of Object detection###
                ### Start of object tracking###
                success,roi=tracker.update(temp_frame) #updating tracker
                (x,y,w,h)=tuple(map(int,roi))
                #Draw rectangle as object moves
                if success:
                    pts1=(x,y)
                    pts2=(x+w,y+h)
                    cv2.rectangle(frame,pts1,pts2,(255,125,25),3)
                    mask=cv2.line(mask,pts1,temp,color,2)
                    frame=cv2.circle(frame,temp,3,color,-1)
                    frame=cv2.add(frame,mask)
                    temp=pts1
                else:
                    cv2.putText(frame,"fail to track the object",(100,200),cv2.FONT_HERSHEY_SIMPLEX,1,(25,125,255),3)
                cv2.putText(frame,tracker_name,(20,400),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,0),3)

                cv2.imshow('object_detection_tracking', frame)

                # Press esc to exit
                if cv2.waitKey(50)&0xFF==27:
                    break
        cap.release()
        cv2.destroyAllWindows()

# Run the below code to start the object detection or tracking or both

In [9]:
choice, tracker=ask_for_input()
if choice=="1":
    object_detection()
elif choice=="2":
    object_tracking(tracker)
elif choice=="3":
    object_detection_tracking(tracker)
else:
    print("enter value inputs")

1. object detection
2. object tracting
3. both object detection and tracking
select any of the beow options3
1. boosting
2. MIL
3. KCF
4. TLD
5. MEDIANFLOW
6. GOTURN
7. MOSSE
8. CSRT
please select your tracker1
